In [9]:
from pathlib import Path

PROTEIN_CLS = "RAS"

WD = Path(f"CR/{PROTEIN_CLS}")
WD.mkdir(parents=True, exist_ok=True)

In [10]:
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
from Bio import SeqIO
from time import sleep
from selenium.webdriver.common.by import By
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC


def consurf(sid):
    # prepare sequences
    seq_path = Path(f"../../data/{PROTEIN_CLS}/generated/Q/{sid}.fasta")
    seqs_path = WD / f"{sid}.fasta"
    seqs = [SeqIO.read(seq_path, "fasta")]
    for _, ref in pd.read_csv(
            f"../../data/{PROTEIN_CLS}_ref.csv"
    ).drop_duplicates(subset=['Entry ID'], keep='first').iterrows():
        seqs.append(SeqRecord(Seq(ref["Sequence"]), id=ref["Entry ID"], description=""))
    with open(seqs_path, "w") as f:
        SeqIO.write(seqs, f, format="fasta")

    # alignment
    aln_path = WD / f"{sid}.aln.fasta"
    !clustalo -i {seqs_path} -o {aln_path} --outfmt=fa

    # run consurf
    pdb_path = Path(f"/home/tz365/WS/PROTEVAL/data/{PROTEIN_CLS}/generated/BQ/{sid}.pdb")

    driver = webdriver.Chrome()
    driver.get('https://consurf.tau.ac.il/consurf_index.php')
    wait = WebDriverWait(driver, 20)

    # PAGE 1
    wait.until(EC.presence_of_element_located((By.ID, 'browse'))).send_keys(str(pdb_path))
    # driver.find_element(By.ID, 'browse').send_keys(str(pdb_path))
    wait.until(EC.presence_of_element_located((By.ID, 'JOB_TITLE'))).send_keys(f"{pdb_path.stem}({PROTEIN_CLS})")
    # driver.find_element(By.ID, 'JOB_TITLE').send_keys(f"{pdb_path.stem}({PROTEIN_CLS})")
    wait.until(EC.presence_of_element_located((By.ID, 'user_email'))).send_keys("ecburxvip@gmail.com")
    # driver.find_element(By.ID, 'user_email').send_keys("ecburxvip@gmail.com")
    sleep(10)
    wait.until(EC.element_to_be_clickable((By.XPATH, "//input[@value='Select Run Parameters Manually']"))).click()
    # driver.find_element(By.XPATH, "//input[@value='Select Run Parameters Manually']").click()

    # PAGE 2
    wait.until(EC.presence_of_element_located((By.ID, 'browse1'))).send_keys(str(aln_path.absolute()))
    sleep(10)
    wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, "div.dropdown-select"))).click()
    wait.until(EC.element_to_be_clickable((By.XPATH, f"//li[@data-value='{sid}']"))).click()
    wait.until(EC.element_to_be_clickable((By.XPATH, "//input[@value='Submit']"))).click()

    # PAGE 3
    url = driver.current_url
    print(f"'{sid}': '{url}',")
    driver.close()
    return url

In [12]:
urls = {
    (sid := f"FM_{i}"): consurf(sid)
    for i in range(0, 50)
}
urls

{'FM_0': None,
 'FM_1': None,
 'FM_2': None,
 'FM_3': None,
 'FM_4': None,
 'FM_5': None,
 'FM_6': None,
 'FM_7': None,
 'FM_8': None,
 'FM_9': None,
 'FM_10': None,
 'FM_11': None,
 'FM_12': None,
 'FM_13': None,
 'FM_14': None,
 'FM_15': None,
 'FM_16': None,
 'FM_17': None,
 'FM_18': None,
 'FM_19': None,
 'FM_20': None,
 'FM_21': None,
 'FM_22': None,
 'FM_23': None,
 'FM_24': None,
 'FM_25': None,
 'FM_26': None,
 'FM_27': None,
 'FM_28': None,
 'FM_29': None,
 'FM_30': None,
 'FM_31': None,
 'FM_32': None,
 'FM_33': None,
 'FM_34': None,
 'FM_35': None,
 'FM_36': None,
 'FM_37': None,
 'FM_38': None,
 'FM_39': None,
 'FM_40': None,
 'FM_41': None,
 'FM_42': None,
 'FM_43': None,
 'FM_44': None,
 'FM_45': None,
 'FM_46': None,
 'FM_47': None,
 'FM_48': None,
 'FM_49': None}

In [ ]:
# urls = {
# }

def retrieve(sid, url):
    driver = webdriver.Chrome()
    driver.get(url)
    driver.find_element(By.LINK_TEXT, "Go To The Results").click()
    sleep(2)
    driver.find_element(By.XPATH, f"//a[contains(@href, '{sid}_consurf_grades.txt')]").click()
    sleep(5)
    driver.close()


for k, v in urls.items():
    retrieve(k, v)

In [ ]:
import pandas as pd
import re
from pathlib import Path

PROTEIN_CLS = "RAS"

Path(f"CR2/{PROTEIN_CLS}").mkdir(parents=True, exist_ok=True)

for src in Path(f"CR/{PROTEIN_CLS}").glob("*.txt"):
    dst = f"CR2/{PROTEIN_CLS}/{src.name}"
    with open(src, 'r', encoding='utf-8') as f:
        lines = f.readlines()
    with open(dst, 'w', encoding='utf-8') as f:
        f.writelines(lines[29:len(lines) - 4])

In [ ]:
Path(f"../../data/{PROTEIN_CLS}/generated/C/").mkdir(parents=True, exist_ok=True)

for path in Path(f"CR2/{PROTEIN_CLS}").glob("*.txt"):
    data = []
    start = False
    with open(path, 'r') as f:
        for line in f:
            parts = line.split()
            pos = parts[0]
            seq = parts[1]
            atom = parts[2]
            score = float(parts[3])
            color = int(re.findall(r'\d+', parts[4])[0])
            confidence_interval = (
                float(parts[5].split(',')[0]), float(parts[6]), *[int(i) for i in parts[7].split(",")])
            be = parts[8]
            if parts[9] == "f" or parts[9] == "s":
                fs = parts[9]
                msa_data = parts[10]
                residue_variety = " ".join(parts[11:])
            else:
                fs = ""
                msa_data = parts[9]
                residue_variety = " ".join(parts[10:])
            data.append([pos, seq, atom, score, color, confidence_interval, be, fs, msa_data, residue_variety])

        pd.DataFrame(
            data,
            columns=[
                "POS", "SEQ", "ATOM", "SCORE", "COLOR",
                "CONFIDENCE_INTERVAL", "B/E", "F/S", "MSA DATA", "RESIDUE_VARIETY"
            ]).to_csv(f"../data/{PROTEIN_CLS}/generated/C/{path.stem}.csv", index=False)